In [1]:
!pip install cmake
!pip install dlib==19.18.0
!pip install face_recognition
!pip install opencv-python

In [2]:
import cv2
import numpy as np
import face_recognition

# Facial Recognition Using DNNs

We integrated a 'Facial Detection Model' as well as a 'Facial Recognition Model'

1. Download and Load the models

In [3]:
face_detector_model = "res10_300x300_ssd_iter_140000.caffemodel"
face_detector_config = "deploy.prototxt"
face_recognition_model = "nn4.small2.v1.t7"

face_detector_net = cv2.dnn.readNetFromCaffe(face_detector_config, face_detector_model) #From Caffe
face_recognizer_net = cv2.dnn.readNetFromTorch(face_recognition_model) #From pyTorch

2. Functions to be used and store the known faces with their labels

In [5]:
def load_known_faces(known_face_paths):
    known_faces = []
    for path, name in known_face_paths:
        image = cv2.imread(path, 1)
        image = cv2.resize(image, (96, 96))
        face_blob = cv2.dnn.blobFromImage(image, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
        face_recognizer_net.setInput(face_blob)
        vec = face_recognizer_net.forward()
        known_faces.append((vec, name))
    return known_faces

def find_closest_match(face_vec, known_faces):
    min_dist = float('inf')
    name = "Unknown"
    for vec, known_name in known_faces:
        dist = np.linalg.norm(face_vec - vec)
        if dist < min_dist:
            min_dist = dist
            name = known_name
    return name

known_face_paths = [
    ("alex.jpeg", "Alex"),
    ("ban.jpeg", "Ban"),
    ("hiba.jpeg", "Hiba")
]
known_faces = load_known_faces(known_face_paths)

3. Start the webcam and run the model

In [6]:
# Start the webcam
cap = cv2.VideoCapture(0)

#Webcam resolution is decreased for more optimized processing (faster)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,640) #
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Run face recognition every 5 frames for efficiency (since large model)
frame_count = 0
face_recognition_interval = 5
last_recognized_faces = {}

while True:
    ret, frame = cap.read()
    if not ret:
        break

    (h, w) = frame.shape[:2]
    if frame_count % face_recognition_interval == 0:
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        face_detector_net.setInput(blob)
        detections = face_detector_net.forward()

        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:
                box = detections[0, 0, i, 3:]*np.array([w,h,w,h])
                (startX,startY,endX,endY)= box.astype("int")
            # Extract the face ROI
        face = frame[startY:endY, startX:endX]
        face_blob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
        face_recognizer_net.setInput(face_blob)
        face_vec = face_recognizer_net.forward()

            # Find the closest match from the known faces
            # Caching results to avoid recalculating for each frame
        name = last_recognized_faces.get((startX, startY, endX, endY))
        if name is None:
            name = find_closest_match(face_vec, known_faces)
            last_recognized_faces[(startX, startY, endX, endY)] = name

        cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 0, 255), 2)
        cv2.putText(frame, name, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

# Display the output
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

frame_count += 1
cap.release()
cv2.destroyAllWindows()

# Facial Recognition Using Python Library

1. Load the images (dataset)

In [7]:
#Load the images that the model will use for recognition:
image_person1 = face_recognition.load_image_file("alex.jpeg")
image_person2 = face_recognition.load_image_file("hiba.jpeg")
image_person3 = face_recognition.load_image_file("ban.jpeg")
image_person4 = face_recognition.load_image_file("ahmad.jpeg")

2. Extract the facial encodings of every person, and name them

  The way a face is represented using a set of numbers:

In [8]:
person1_face_encoding = face_recognition.face_encodings(image_person1)[0]
person2_face_encoding = face_recognition.face_encodings(image_person2)[0]
person3_face_encoding = face_recognition.face_encodings(image_person3)[0]
person4_face_encoding = face_recognition.face_encodings(image_person4)[0]

known_face_encodings = [person1_face_encoding,person2_face_encoding,person3_face_encoding,person4_face_encoding]
known_face_names = ["Alex", "Hiba","Ban","Ahla Dr.Ahmad"]

3. Run the code by detecting face locations and their encodings and labelling them accordingly

In [9]:
face_locations = []
face_encodings = []
face_names = []

#To process every other frame for efficiency
process_this_frame = True


video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()

    # Resize frame for faster face recognition processing ---> detection on different scale
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR (which OpenCV uses) to RGB (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Process every other frame to save time (skips 1 frame)
    if process_this_frame:
        # Find all the faces and face encodings in the current frame
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:

            # See if the face is a match for the known faces
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # Use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame #to skip one frame, it turns this parameter to false

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size (original image)
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a rectangle around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


video_capture.release()
cv2.destroyAllWindows()

# References:
DNN:

    - Facial Detection Model:

    https://github.com/sr6033/face-detection-with-OpenCV-and-DNN/blob/master/detect_faces_video.py

    - Facial Recognition Model:

    https://cmusatyalab.github.io/openface/models-and-accuracies/
    
    http://gitlab.ubocare.com/jiajunjie/face_detect/blob/5891825ac2f2acd8ad8ac31093f1048f4197f722/models/face_detector/openface.nn4.small2.v1.t7

Facial Recognition Application:

    https://pypi.org/project/face-recognition/